In [1]:
import pandas as pd

In [4]:
df=pd.read_pickle("/workspace/LLaRA/data/ref/movielens/similar_test_data.df")

In [5]:
df.head()

,seq,len_seq,next,movie_names_only,seq_only,movie_embeddings,most_similar_seq_index,most_similar_seq,most_similar_seq_next,most_similar_seq_name,most_similar_seq_next_name
0,"[(704, 5), (155, 3), (80, 4), (698, 4), (745, ...",10,"(3, 4)","[House of the Spirits, The (1993), Dirty Danci...","[704, 155, 80, 698, 745, 95, 1403, 202, 731, 432]","[0.01653142, 0.06018769, 0.12848003, -0.023819...",4,"[(11, 5), (198, 5), (192, 5), (184, 5), (640, ...","(340, 3)","[Seven (Se7en) (1995), Nikita (La Femme Nikita...",Boogie Nights (1997)
1,"[(1305, 5), (135, 4), (1125, 5), (1124, 5), (2...",10,"(132, 5)","[National Lampoon's Senior Trip (1995), 2001: ...","[1305, 135, 1125, 1124, 240, 605, 477, 485, 52...","[0.002688799, -0.0696835, 0.14252476, -0.01397...",17,"[(801, 4), (529, 3), (448, 5), (569, 4), (230,...","(585, 3)","[Air Up There, The (1994), My Life as a Dog (M...",Son in Law (1993)
2,"[(1030, 5), (394, 3), (104, 3), (779, 4), (103...",10,"(671, 3)","[Beverly Hillbillies, The (1993), Radioland Mu...","[1030, 394, 104, 779, 1036, 66, 446, 1029, 451...","[-0.014814284, 0.00812782, 0.038343, 0.0050779...",29,"[(258, 3), (69, 5), (735, 5), (1021, 4), (268,...","(257, 5)","[Contact (1997), Forrest Gump (1994), Philadel...",Men in Black (1997)
3,"[(537, 4), (293, 4), (244, 3), (287, 5), (524,...",10,"(21, 5)","[My Own Private Idaho (1991), Donnie Brasco (1...","[537, 293, 244, 287, 524, 514, 922, 602, 172, 97]","[0.0041796635, -0.13311706, 0.06252247, 0.0116...",36,"[(149, 4), (509, 4), (323, 4), (461, 4), (960,...","(1524, 4)","[Jude (1996), My Left Foot (1989), Dante's Pea...",Kaspar Hauser (1993)
4,"[(11, 5), (198, 5), (192, 5), (184, 5), (640, ...",10,"(340, 3)","[Seven (Se7en) (1995), Nikita (La Femme Nikita...","[11, 198, 192, 184, 640, 581, 653, 432, 356, 666]","[-0.16389114, -0.076835975, 0.05094645, 0.1335...",50,"[(6, 3), (673, 5), (217, 4), (163, 4), (671, 3...","(86, 4)",[Shanghai Triad (Yao a yao yao dao waipo qiao)...,"Remains of the Day, The (1993)"


In [6]:
import torch
import os.path as op
import numpy as np
import pickle as pkl
import torch.utils.data as data

import pandas as pd
import random

class MovielensData(data.Dataset):
    def __init__(self, data_dir=r'data/ref/movielens',
                 stage=None,
                 cans_num=10,
                 sep=", ",
                 no_augment=True):
        self.__dict__.update(locals())
        self.aug = (stage=='train') and not no_augment
        self.padding_item_id=1682
        self.padding_rating=0
        self.check_files()

    def __len__(self):
        return len(self.session_data['seq'])

    def __getitem__(self, i):
        temp = self.session_data.iloc[i]
        candidates = self.negative_sampling(temp['seq_unpad'],temp['next'])
        cans_name=[self.item_id2name[can] for can in candidates]
        sample = {
            'seq': temp['seq'],
            'seq_name': temp['seq_title'],
            'len_seq': temp['len_seq'],
            'seq_str': self.sep.join(temp['seq_title']),
            'cans': candidates,
            'cans_name': cans_name,
            'cans_str': self.sep.join(cans_name),
            'len_cans': self.cans_num,
            'item_id': temp['next'],
            'item_name': temp['next_item_name'],
            'correct_answer': temp['next_item_name'],
            'most_similar_seq': temp['most_similar_seq'],
            'most_similar_seq_next': temp['most_similar_seq_next'],
            'most_similar_seq_name': temp['most_similar_seq_name'],
            'most_similar_seq_next_name': temp['most_similar_seq_next_name'],
            
        }
        return sample
    
    def negative_sampling(self,seq_unpad,next_item):
        canset=[i for i in list(self.item_id2name.keys()) if i not in seq_unpad and i!=next_item]
        candidates=random.sample(canset, self.cans_num-1)+[next_item]
        random.shuffle(candidates)
        return candidates  

    def check_files(self):
        self.item_id2name=self.get_movie_id2name()
        if self.stage=='train':
            filename="similar_train_data.df"
        elif self.stage=='val':
            filename="similar_val_data.df"
        elif self.stage=='test':
            filename="similar_test_data.df"
        data_path=op.join(self.data_dir, filename)
        self.session_data = self.session_data4frame(data_path, self.item_id2name)  
    
    def get_mv_title(self,s):
        sub_list=[", The", ", A", ", An"]
        for sub_s in sub_list:
            if sub_s in s:
                return sub_s[2:]+" "+s.replace(sub_s,"")
        return s

    def get_movie_id2name(self):
        movie_id2name = dict()
        item_path=op.join(self.data_dir, 'u.item')
        with open(item_path, 'r', encoding = "ISO-8859-1") as f:
            for l in f.readlines():
                ll = l.strip('\n').split('|')
                movie_id2name[int(ll[0]) - 1] = self.get_mv_title(ll[1][:-7])
        return movie_id2name
    
    def session_data4frame(self, datapath, movie_id2name):
        train_data = pd.read_pickle(datapath)
        train_data = train_data[train_data['len_seq'] >= 3]
        def remove_padding(xx):
            x = xx[:]
            for i in range(10):
                try:
                    x.remove((self.padding_item_id,self.padding_rating))
                except:
                    break
            return x
        train_data['seq_unpad'] = train_data['seq'].apply(remove_padding)
        def seq_to_title(x): 
            return [movie_id2name[x_i[0]] for x_i in x]
        train_data['seq_title'] = train_data['seq_unpad'].apply(seq_to_title)
        def next_item_title(x): 
            return movie_id2name[x[0]]
        train_data['next_item_name'] = train_data['next'].apply(next_item_title)
        def get_id_from_tumple(x):
            return x[0]
        def get_id_from_list(x):
            return [i[0] for i in x]

        train_data['next'] = train_data['next'].apply(get_id_from_tumple)
        train_data['seq'] = train_data['seq'].apply(get_id_from_list)
        train_data['seq_unpad']=train_data['seq_unpad'].apply(get_id_from_list)
        train_data['most_similar_seq_next'] = train_data['most_similar_seq_next'].apply(get_id_from_tumple)
        train_data['most_similar_seq'] = train_data['most_similar_seq'].apply(get_id_from_tumple)
       

        return train_data
        



In [21]:
from torch.utils.data import DataLoader

# 假设数据目录和阶段已经被正确设置
dataset = MovielensData(data_dir='/workspace/LLaRA/data/ref/movielens', stage='train')
print(dataset)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)  # 设置批量大小为10

# 获取一个批次的数据并打印
for batch in dataloader:
    # print(batch)
    
    # 检查 'cans' 是否包含 'correct_answer'
    cans = batch['cans_name']
    correct_answer = batch['correct_answer']
    
    if correct_answer in cans:
        print(f"'correct_answer' {correct_answer} is in 'cans_name': {cans_name}")
    else:
        print(f"'correct_answer' {correct_answer} is NOT in 'cans_name': {cans_name}")
    
    break  # 只打印第一个批次


'correct_answer' ['Gigi'] is NOT in 'cans_name': [('He Walked by Night',), ('Legends of the Fall',), ('Searching for Bobby Fischer',), ('Cold Comfort Farm',), ('Colonel Chabert, Le',), ('Get Shorty',), ('Amateur',), ('Rosencrantz and Guildenstern Are Dead',), ('Turbulence',), ('The Haunted World of Edward D. Wood Jr.',)]


In [ ]:
from torch.utils.data import DataLoader

# 假设数据目录和阶段已经被正确设置
dataset = MovielensData(data_dir='/workspace/LLaRA/data/ref/movielens', stage='train')
print(dataset)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)  # 设置批量大小为10

# 检查整个测试数据集中的每个批次
correct_in_cans_name_count = 0
total_batches = 0

for batch in dataloader:
    # 将cans_name中的元组展开为字符串列表
    cans_name_list = [item[0] for item in batch['cans_name']]
    correct_answer = batch['correct_answer'][0]  # 假设correct_answer是单个元素的列表
    
    if correct_answer in cans_name_list:
        correct_in_cans_name_count += 1
    #     print(f"'correct_answer' {correct_answer} is in 'cans_name': {cans_name_list}")
    # else:
    #     print(f"'correct_answer' {correct_answer} is NOT in 'cans_name': {cans_name_list}")
    
    total_batches += 1

print(f"Total batches: {total_batches}")
print(f"Batches where 'correct_answer' is in 'cans_name': {correct_in_cans_name_count}")
print(f"Percentage: {correct_in_cans_name_count / total_batches * 100:.2f}%")
